In [ ]:
# ============================================================
# NOTEBOOK 1 — GENERACIÓN DE ARCHIVOS BASE PARA POWER BI
# Proyecto SABER 11 (2015–2026)
# Autor: John Jairo Prado Piñeres
# Versión estable (2025)
# ============================================================

In [1]:
import os
import pandas as pd


In [6]:
# RUTAS FINALES
ruta_base = r"C:/Users/john/Desktop/modelo-espaciotemporal-saber11"
ruta_entrada = ruta_base + "/data/4_modelado"
ruta_salida  = ruta_base + "/resultados/powerbi"

# Crear carpeta de salida
os.makedirs(ruta_salida, exist_ok=True)

print("Rutas configuradas")
print(" - Entrada:", ruta_entrada)
print(" - Salida :", ruta_salida)

Rutas configuradas
 - Entrada: C:/Users/john/Desktop/modelo-espaciotemporal-saber11/data/4_modelado
 - Salida : C:/Users/john/Desktop/modelo-espaciotemporal-saber11/resultados/powerbi


In [ ]:
# Cargar automáticamente todos los archivos impXXXX_db1_completo.csv

In [7]:
# Cargar todos los CSV dentro de 4_modelado
archivos = sorted([f for f in os.listdir(ruta_entrada) if f.endswith(".csv")])

dfs = []
for archivo in archivos:
    if archivo.startswith("imp") and "completo" in archivo:
        anio = int(archivo.replace("imp","").replace("_db1_completo.csv",""))
        df = pd.read_csv(os.path.join(ruta_entrada, archivo))
        df["anio"] = anio
        dfs.append(df)

df_total = pd.concat(dfs, ignore_index=True)
print("Base consolidada:", df_total.shape)
df_total.head()


Base consolidada: (2851299, 53)


,PERIODO,ESTU_TIPODOCUMENTO,ESTU_CONSECUTIVO,COLE_AREA_UBICACION,COLE_BILINGUE,COLE_CALENDARIO,COLE_CARACTER,COLE_COD_DANE_ESTABLECIMIENTO,COLE_COD_DANE_SEDE,COLE_COD_DEPTO_UBICACION,...,FAMI_TIENELAVADORA,DESEMP_INGLES,PUNT_INGLES,PUNT_MATEMATICAS,PUNT_SOCIALES_CIUDADANAS,PUNT_C_NATURALES,PUNT_LECTURA_CRITICA,PUNT_GLOBAL,ANIO,anio
0,20151,TI,SB11201510000182,URBANO,S,B,ACADÉMICO,3.080010e+11,3.080010e+11,8.0,...,Si,B+,90.0,58,62,63,77,335,2015,2015
1,20151,TI,SB11201510000183,URBANO,S,B,ACADÉMICO,3.080010e+11,3.080010e+11,8.0,...,Si,A2,68.0,62,53,64,64,307,2015,2015
2,20151,TI,SB11201510000207,URBANO,N,B,ACADÉMICO,3.190010e+11,3.190010e+11,19.0,...,Si,A1,53.0,61,60,55,58,290,2015,2015
3,20151,TI,SB11201510000208,URBANO,N,B,ACADÉMICO,3.190010e+11,3.190010e+11,19.0,...,Si,B1,76.0,86,77,74,69,382,2015,2015
4,20151,TI,SB11201510000209,URBANO,N,B,ACADÉMICO,3.190010e+11,3.190010e+11,19.0,...,Si,A2,66.0,79,60,76,55,337,2015,2015


In [ ]:
# Normalizar nombres de columnas

In [8]:
# Cambiar nombres de columnas según el ICFES
df_total = df_total.rename(columns={
    "COLE_DEPTO_UBICACION": "departamento",
    "COLE_DEPTO": "departamento",
    "ESTU_DEPTO_PRESENTACION": "departamento_presentacion",
    "ESTU_DEPTO_RESIDE": "departamento_residencia",
    "PUNT_GLOBAL": "puntaje_global",
})


In [ ]:
# Puntajes por DEPARTAMENTO (2015–2022)

In [9]:
puntajes_dep = (
    df_total
    .groupby(["anio", "departamento"])["puntaje_global"]
    .mean()
    .reset_index()
)

puntajes_dep.rename(columns={"puntaje_global": "puntaje_promedio"}, inplace=True)
puntajes_dep.head()


,anio,departamento,puntaje_promedio
0,2015,AMAZONAS,220.590085
1,2015,ANTIOQUIA,247.540718
2,2015,ARAUCA,246.008498
3,2015,ATLANTICO,245.708802
4,2015,BOGOTA,270.212909


In [ ]:
# Puntaje NACIONAL por año (2015–2022)

In [10]:
puntajes_nal = (
    df_total.groupby("anio")["puntaje_global"]
    .mean()
    .reset_index()
)

puntajes_nal.rename(columns={"puntaje_global": "puntaje_promedio"}, inplace=True)
puntajes_nal


,anio,puntaje_promedio
0,2015,250.976849
1,2016,259.792137
2,2017,256.614012
3,2018,287.749874
4,2019,247.827254
5,2020,289.434338
6,2021,301.820325
7,2022,251.387507


In [ ]:
# Crear tabla_departamentos

In [11]:
tabla_departamentos = pd.DataFrame({
    "departamento": [
        "Antioquia","Atlántico","Bogotá D.C.","Bolívar","Boyacá","Caldas","Caquetá","Cauca",
        "Cesar","Córdoba","Cundinamarca","Chocó","Huila","La Guajira","Magdalena","Meta",
        "Nariño","Norte de Santander","Quindío","Risaralda","Santander","Sucre","Tolima",
        "Valle del Cauca","Arauca","Casanare","Putumayo","San Andrés","Amazonas","Guaviare",
        "Guainía","Vaupés"
    ],
    "cod_dane_depto": [
        5,8,11,13,15,17,18,19,20,23,25,27,41,44,47,50,52,
        54,63,66,68,70,73,76,81,85,86,88,91,94,95,97
    ]
})


In [ ]:
# Exportar todos los archivos para Power BI

In [12]:
# Exportar archivos
puntajes_dep.to_csv(os.path.join(ruta_salida, "puntajes_departamento_2015_2022.csv"), index=False)
puntajes_nal.to_csv(os.path.join(ruta_salida, "puntajes_nacionales_2015_2022.csv"), index=False)
puntajes_nal.to_csv(os.path.join(ruta_salida, "serie_puntajes_2015_2022.csv"), index=False)
tabla_departamentos.to_csv(os.path.join(ruta_salida, "tabla_departamentos.csv"), index=False)

print("\n ARCHIVOS EXPORTADOS PARA POWER BI:")
print("  - puntajes_departamento_2015_2022.csv")
print("  - puntajes_nacionales_2015_2022.csv")
print("  - serie_puntajes_2015_2022.csv")
print("  - tabla_departamentos.csv")
print("\nNotebook 1 ejecutado exitosamente.")



 ARCHIVOS EXPORTADOS PARA POWER BI:
  - puntajes_departamento_2015_2022.csv
  - puntajes_nacionales_2015_2022.csv
  - serie_puntajes_2015_2022.csv
  - tabla_departamentos.csv

Notebook 1 ejecutado exitosamente.
